In [22]:
import os
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from llama_index.query_engine import CitationQueryEngine
from llama_index.retrievers import VectorIndexRetriever
from llama_index import (
    VectorStoreIndex,
    ResponseSynthesizer,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
    LLMPredictor,
    ServiceContext,
)
import textwrap

In [2]:
service_context = ServiceContext.from_defaults(
    llm_predictor=LLMPredictor(llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0))
)

In [16]:

documents = SimpleDirectoryReader("../data/raw/documents/").load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
index.storage_context.persist(persist_dir="./citation")


In [32]:
query_engine = CitationQueryEngine.from_args(
    index, 
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=1024 
)

In [33]:
response = query_engine.query("Cuales son las funciones del presidente?")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


In [25]:
response.response

'El presidente de la República tiene varias funciones y atribuciones especiales, según el Artículo 32 de la Constitución de Chile [1]. Algunas de estas funciones incluyen: concurrir a la formación de las leyes con arreglo a la Constitución, sancionarlas y promulgarlas; pedir que se cite a sesión a cualquiera de las ramas del Congreso Nacional; dictar decretos con fuerza de ley sobre las materias que señala la Constitución; convocar a plebiscito en los casos del artículo 128; declarar los estados de excepción constitucional en los casos y formas que se señalan en esta Constitución; ejercer la potestad reglamentaria en todas aquellas materias que no sean propias del dominio legal, sin perjuicio de la facultad de dictar los demás reglamentos, decretos e instrucciones que crea convenientes para la ejecución de las leyes; nombrar y remover a su voluntad a los ministros de Estado, subsecretarios, delegados presidenciales regionales y delegados presidenciales provinciales; designar a los emba

In [34]:
print(textwrap.fill(response.response, width=100))

El presidente de la República tiene diversas funciones y atribuciones especiales, según lo
establecido en el Artículo 32 de la Constitución chilena. Entre ellas se encuentran: concurrir a la
formación de las leyes, pedir que se cite a sesión a cualquiera de las ramas del Congreso Nacional,
dictar decretos con fuerza de ley sobre las materias que señala la Constitución, convocar a
plebiscito en los casos del artículo 128, declarar los estados de excepción constitucional, ejercer
la potestad reglamentaria, nombrar y remover a su voluntad a los ministros de Estado,
subsecretarios, delegados presidenciales regionales y delegados presidenciales provinciales,
designar a los embajadores y ministros diplomáticos, y a los representantes ante organismos
internacionales, nombrar al Contralor General de la República con acuerdo del Senado, nombrar y
remover a los funcionarios que la ley denomina como de su exclusiva confianza y proveer los demás
empleos civiles en conformidad a la ley, conceder ju

In [41]:
response = query_engine.query("Existen los extraterrestres?")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


In [42]:
print(textwrap.fill(response.response, width=100))

None of the provided sources address the existence of extraterrestrials.


In [43]:
response = query_engine.query("Cual es el rol de la Constitución?")

In [44]:
print(textwrap.fill(response.response, width=100))

La Constitución establece las bases de la institucionalidad y regula el ejercicio de la soberanía
por parte del pueblo y las autoridades [2]. Los órganos del Estado deben someter su acción a la
Constitución y a las normas dictadas conforme a ella, y garantizar el orden institucional de la
República [3]. La Constitución también establece normas generales y obligatorias en diversos
ámbitos, como delitos contemplados en el artículo 9º, residencia del Presidente de la República,
funcionamiento de la Corte Suprema y el Tribunal Constitucional, procedimientos de la administración
pública, y regulación de loterías, hipódromos y apuestas en general [1].


In [46]:
print(response.source_nodes[2].node.get_text())

Source 3:
Capítulo I: BASES DE LA INSTITUCIONALIDAD
Artículo 6º
Los órganos del Estado deben someter su acción a la Constitución y a las normas dictadas conforme a ella, y garantizar el orden institucional de la República. Los preceptos de esta Constitución obligan tanto a los titulares o integrantes de dichos órganos como a toda persona, institución o grupo. La infracción de esta norma generará las responsabilidades y sanciones que determine la ley.



In [30]:
response.source_nodes[1].node

Node(text='Source 2:\n- Nombrar al Contralor General de la República con acuerdo del Senado;\n10º.- Nombrar y remover a los funcionarios que la ley denomina como de su exclusiva confianza y proveer los demás empleos civiles en conformidad a la ley. La remoción de los demás funcionarios se hará de acuerdo a las disposiciones que ésta determine;\n11º.- Conceder jubilaciones, retiros, montepíos y pensiones de gracia, con arreglo a las leyes;\n12º.- Nombrar a los magistrados y fiscales judiciales de las Cortes de Apelaciones y a los jueces letrados, a proposición de la Corte Suprema y de las Cortes de Apelaciones, respectivamente; a los miembros del Tribunal Constitucional que le corresponde designar; y a los magistrados y fiscales judiciales de la Corte Suprema y al Fiscal Nacional, a proposición de dicha Corte y con acuerdo del Senado, todo ello conforme a lo prescrito en esta Constitución;\n13º.- Velar por la conducta ministerial de los jueces y demás empleados del Poder Judicial y requ